# Descriptors

walking through the official docs at https://docs.python.org/3/howto/descriptor.html#descriptor-howto-guide

## A trivial exampe

In [3]:
import time

class Ten:
    def __get__(self, obj, objtype=None):
        time.sleep(3)
        return 10 

In [4]:
class A:
    x = 5
    y = Ten()

In [5]:
a = A()

In [6]:
a.x  # from class dictionary

5

In [7]:
a.y  # attribute value is not stored, but computed on demand

10


## Dynamic lookup

In [8]:
import os

class DirectorySize:
    def __get__(self, obj, objtype=None):
        print(obj)
        print(objtype)
        return len(os.listdir(obj.dirname))
    
class Directory:
    size = DirectorySize()
    
    def __init__(self, dirname):
        self.dirname = dirname

Parameters to `__get__`

* size` is an instance of `DirectorySize`
* obj an instance of Directory
* objtype is the Directory class

In [9]:
d = Directory('.')

In [10]:
d.size

<class '__main__.Directory'>


12

In [11]:
d.dirname

'.'

In [12]:
os.listdir(d.dirname)

['refreeze',
 'descriptors.ipynb',
 'requirements.txt',
 '.ipynb_checkpoints',
 'talk.md',
 '.gitmodules',
 '.gitignore',
 '.venv',
 'talk.css',
 'index.html',
 'Makefile',
 '.git']

In [13]:
len(os.listdir(d.dirname))

12

## Managed attributes

* public attribute has descriptor
* private attribute has actual data

### Logging attribute access

In [14]:
class LoggedAgeAccess:
    def __get__(self, obj, objtype=None):
        print('foo')
        return obj._age
    
    def __set__(self, obj, value):
        print('bar')
        obj._age = value

class Person:
    age = LoggedAgeAccess()
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def birthday(self):
        self.age += 1

In [15]:
mary = Person('Mary', 70)

bar


In [16]:
mary.name

'Mary'

In [17]:
mary.age

foo


70

In [18]:
vars(mary)

{'name': 'Mary', '_age': 70}

In [19]:
dir(mary)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_age',
 'age',
 'birthday',
 'name']

In [22]:
mary.birthday()

foo
bar


## Customized names

In [24]:
class LoggedAccess:
    
    def __set_name__(self, owner, name):
        self.public_name = name
        self.private_name = f'_{name}'
        
    def __get__(self, obj, objtype=None):
        value = getattr(obj, self.private_name)
        print('foo')
        
    def __set__(self, obj, value):
        print('bar')
        setattr(obj, self.private_name, value)
        
class Person:
    name = LoggedAccess()
    age = LoggedAccess()
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def birthday(self):
        self.age += 1

In [25]:
Person

__main__.Person

In [26]:
vars(Person)

mappingproxy({'__module__': '__main__',
              'name': <__main__.LoggedAccess at 0x7fa6f46a90a0>,
              'age': <__main__.LoggedAccess at 0x7fa6f46a91c0>,
              '__init__': <function __main__.Person.__init__(self, name, age)>,
              'birthday': <function __main__.Person.birthday(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [27]:
vars(Person)['name']

In [28]:
vars(vars(Person)['name']) # look up descriptor without triggering it

{'public_name': 'name', 'private_name': '_name'}

In [29]:
vars(vars(Person)['age'])

{'public_name': 'age', 'private_name': '_age'}

In [30]:
pete = Person('Peter', 10)

bar
bar


In [31]:
kate = Person('Catherine', 20)

bar
bar


In [32]:
vars(pete)

{'_name': 'Peter', '_age': 10}

In [33]:
vars(kate)

{'_name': 'Catherine', '_age': 20}

* A descriptor is an object that defines `__get__`, `__set__`, also `__delete__`
* `__set_name__` is needed when the descriptor needs to know about the class or class variable
* Invoked by dot operator attribute lookup
* Need to be used as class variables
* Motivation: provide a hook for controlling what happens during a dot lookup
* Removes control from class definition to data being looked up
* Examples which are implemented as descriptors
    - `classmethod`
    - `staticmethod`
    - `property`
    - `functools.cached_property`

## Complete Practical Example

### Validator class

A descriptor for managed attribute access

In [71]:
import abc

class Validator(abc.ABC):                             # an abstract base class
    def __set_name__(self, owner, name):
        self.private_name = f'_{name}'
        
    def __get__(self, obj, objtype=None):
        return getattr(obj, self.private_name)        #  a descriptor
    
    def __set__(self, obj, value):
        self.validate(value)
        setattr(obj, self.private_name, value)
        
    @abc.abstractmethod
    def validate(self):
        pass

### verify that a value is one of a series of options

In [72]:
class OneOf(Validator):

    def __init__(self, *options):
        self.options = set(options)
        
    def validate(self, value):
        if value not in self.options:
            raise ValueError(f'Expected {value} to be one of {self.options}')

In [74]:
class Foo:
    value = OneOf(1, 2)
    
    def __init__(self, value=1):
        self.value = value

In [75]:
foo = Foo()

In [76]:
foo.value

1

In [77]:
foo.value = 2


In [78]:
foo.value

2

In [85]:
# foo.value = 3

    foo.value = 3
    
    ---------------------------------------------------------------------------
    ValueError                                Traceback (most recent call last)
    <ipython-input-79-225e3a6b882d> in <module>
    ----> 1 foo.value = 3

    <ipython-input-71-46f3483445db> in __set__(self, obj, value)
          9 
         10     def __set__(self, obj, value):
    ---> 11         self.validate(value)
         12         setattr(obj, self.private_name, value)
         13 

    <ipython-input-72-39812059dd46> in validate(self, value)
          6     def validate(self, value):
          7         if value not in self.options:
    ----> 8             raise ValueError(f'Expected {value} to be one of {self.options}')

    ValueError: Expected 3 to be one of {1, 2}



### Number and range validator

In [108]:
class Number(Validator):
    
    def __init__(self, minvalue=None, maxvalue=None):
        self.minvalue = minvalue
        self.maxvalue = maxvalue
        
    def validate(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError(f'{value} has wrong type')
            
        if self.minvalue is not None:
            assert value >= self.minvalue, f'{value} too small'
            
        if self.maxvalue is not None:
            assert value <= self.maxvalue, f'{value} too large'
            
        

In [109]:
class Foo:
    value = Number()

In [110]:
foo = Foo()

In [111]:
foo.value = 1

In [112]:
# foo.value = 'bar'

    TypeError                                 Traceback (most recent call last)
    <ipython-input-93-206f4cfd8d6e> in <module>
    ----> 1 foo.value = 'bar'

    <ipython-input-71-46f3483445db> in __set__(self, obj, value)
          9 
         10     def __set__(self, obj, value):
    ---> 11         self.validate(value)
         12         setattr(obj, self.private_name, value)
         13 

    <ipython-input-86-3bac73ce19d8> in validate(self, value)
          7     def validate(self, value):
          8         if not isinstance(value, (int, float)):
    ----> 9             raise TypeError(f'{value} has wrong type')
         10 
         11         if self.minvalue is not None:

    TypeError: bar has wrong type


In [113]:
class Bar:
    value = Number(1, 3)

In [114]:
bar = Bar()

In [115]:
bar.value = 1

In [117]:
# bar.value = 0


    ---------------------------------------------------------------------------
    AssertionError                            Traceback (most recent call last)
    <ipython-input-107-c136dd53065f> in <module>
    ----> 1 bar.value = 0

    <ipython-input-71-46f3483445db> in __set__(self, obj, value)
          9 
         10     def __set__(self, obj, value):
    ---> 11         self.validate(value)
         12         setattr(obj, self.private_name, value)
         13 

    <ipython-input-86-3bac73ce19d8> in validate(self, value)
         10 
         11         if self.minvalue is not None:
    ---> 12             assert value >= self.minvalue
         13 
         14         if self.maxvalue is not None:

    AssertionError: 0 too small


In [119]:
# bar.value = 4

---------------------------------------------------------------------------
    AssertionError                            Traceback (most recent call last)
    <ipython-input-98-c136dd53065f> in <module>
    ----> 1 bar.value = 0

    <ipython-input-71-46f3483445db> in __set__(self, obj, value)
          9 
         10     def __set__(self, obj, value):
    ---> 11         self.validate(value)
         12         setattr(obj, self.private_name, value)
         13 

    <ipython-input-86-3bac73ce19d8> in validate(self, value)
         10 
         11         if self.minvalue is not None:
    ---> 12             assert value >= self.minvalue
         13 
         14         if self.maxvalue is not None:

    AssertionError: 4 too large


### String validator

In [143]:
class String(Validator):
    def __init__(self, minsize=None, maxsize=None, predicate=None):
        self.minsize = minsize
        self.maxsize = maxsize
        self.predicate = predicate
        
    def validate(self, value):
        assert isinstance(value, str), 'Wrong type'
        if self.minsize is not None:
            assert len(value) >= self.minsize, 'Too short'
        if self.maxsize is not None:
            assert len(value) <= self.maxsize, 'Too long'
        if self.predicate is not None:
            assert self.predicate(value)
        

In [156]:
class Foo:
    string = String(1, 6, lambda s: not s.startswith('foo'))

In [157]:
foo = Foo()

In [158]:
foo.string = 'bar'

In [159]:
# foo.string = 1

    ---------------------------------------------------------------------------
    AssertionError                            Traceback (most recent call last)
    <ipython-input-141-b691054e049f> in <module>
    ----> 1 foo.string = 1

    <ipython-input-71-46f3483445db> in __set__(self, obj, value)
          9 
         10     def __set__(self, obj, value):
    ---> 11         self.validate(value)
         12         setattr(obj, self.private_name, value)
         13 

    <ipython-input-137-f7279bd8de7d> in validate(self, value)
          6 
          7     def validate(self, value):
    ----> 8         assert isinstance(value, str)
          9         if self.minsize is not None:
         10             assert len(value) >= self.minsize, 'Too short'

    AssertionError:  Wrong type


In [160]:
# foo.string = ''

    ---------------------------------------------------------------------------
    AssertionError                            Traceback (most recent call last)
    <ipython-input-149-236f25d3f97b> in <module>
    ----> 1 foo.string = ''

    <ipython-input-71-46f3483445db> in __set__(self, obj, value)
          9 
         10     def __set__(self, obj, value):
    ---> 11         self.validate(value)
         12         setattr(obj, self.private_name, value)
         13 

    <ipython-input-143-150b151ed35c> in validate(self, value)
          8         assert isinstance(value, str), 'Wrong type'
          9         if self.minsize is not None:
    ---> 10             assert len(value) >= self.minsize, 'Too short'
         11         if self.maxsize is not None:
         12             assert len(value) <= self.maxsize, 'Too long'

    AssertionError: Too short



In [163]:
# foo.string = 'foobarbaz'

    ---------------------------------------------------------------------------
    AssertionError                            Traceback (most recent call last)
    <ipython-input-151-f0bcb1d4aadb> in <module>
    ----> 1 foo.string = 'foobarbaz'

    <ipython-input-71-46f3483445db> in __set__(self, obj, value)
          9 
         10     def __set__(self, obj, value):
    ---> 11         self.validate(value)
         12         setattr(obj, self.private_name, value)
         13 

    <ipython-input-143-150b151ed35c> in validate(self, value)
         10             assert len(value) >= self.minsize, 'Too short'
         11         if self.maxsize is not None:
    ---> 12             assert len(value) <= self.maxsize, 'Too long'
         13         if self.predicate is not None:
         14             assert self.predicate(value)

    AssertionError: Too long


In [164]:
# foo.string = 'foobar'


    ---------------------------------------------------------------------------
    AssertionError                            Traceback (most recent call last)
    <ipython-input-165-853dbd1aeedf> in <module>
    ----> 1 foo.string = 'foobar'

    <ipython-input-71-46f3483445db> in __set__(self, obj, value)
          9 
         10     def __set__(self, obj, value):
    ---> 11         self.validate(value)
         12         setattr(obj, self.private_name, value)
         13 

    <ipython-input-143-150b151ed35c> in validate(self, value)
         12             assert len(value) <= self.maxsize, 'Too long'
         13         if self.predicate is not None:
    ---> 14             assert self.predicate(value)
         15 

    AssertionError: 


### Practical use case

In [166]:
class Component:

    name = String(minsize=3, maxsize=10, predicate=str.isupper)
    kind = OneOf('wood', 'metal', 'plastic')
    quantity = Number(minvalue=0)

    def __init__(self, name, kind, quantity):
        self.name = name
        self.kind = kind
        self.quantity = quantity


In [168]:
Component('BLA', 'wood', 1)